## Scala传名参数和传值参数
### 定义
Scala的解释器在解析函数参数(function arguments)时有两种方式：  
* 先计算参数表达式的值(reduce the arguments)，再应用到函数内部；或者是将未计算的参数表达式直接应用到函数内部。  
* 前者叫做传值调用（call-by-value），后者叫做传名调用（call-by-name）。  

例如：
```scala
object Add {  
  def addByName(a: Int, b: => Int) = a + b   
  def addByValue(a: Int, b: Int) = a + b   
}
```
addByName是传名调用，addByValue是传值调用。语法上可以看出，使用传名调用时，在参数名称和参数类型中间有一个" =>"符号。

以a为2，b为2+2为例，他们在Scala解释器进行参数规约（reduction）时的顺序分别是这样的：
```scala
    addByName(2, 2 + 2)  
->2 + (2 + 2)  
->2 + 4  
->6  
    
    addByValue(2, 2 + 2)  
->addByValue(2, 4)  
->2 + 4  
->6

```
可以看出，在进入函数内部前，传值调用方式就已经将参数表达式的值计算完毕，而传名调用是在函数内部进行参数表达式的值计算的。
这就造成了一种现象，每次使用传名调用时，解释器都会计算一次表达式的值。对于有副作用(side-effect)的参数来说，这无疑造成了两种调用方式结果的不同。

### 两者的比较
* 传值调用在进入函数体之前就对参数表达式进行了计算，这避免了函数内部多次使用参数时重复计算其值，在一定程度上提高了效率。
* 传名调用的一个优势在于，如果参数在函数体内部没有被使用到，那么它就不用计算参数表达式的值了。在这种情况下，传名调用的效率会高一点。

以一个具体的例子来说明传名参数的用法：
```scala
    var assertionsEnabled = true
    def myAssert(predicate: => Boolean ) = {
        if(assertionsEnabled && !predicate)
        throw new AssertionError
    }
```
这个myAssert函数的参数为一个函数类型，如果标志assertionsEnabled为True时，mymyAssert 根据predicate 的真假决定是否抛出异常，如果assertionsEnabled 为false,则这个函数什么也不做。

现在可以直接使用下面的语法来调用myNameAssert：
```scala
    myNameAssert(5 > 3)
```

看到这里，你可能会想我为什么不直接把参数类型定义为Boolean，比如：
```scala
    def boolAssert(predicate: Boolean ) = {
        if(assertionsEnabled && !predicate)
        throw new AssertionError
    }
```

它的调用也可以使用boolAssert(5 > 3)，和myNameAssert 调用看起来也没什么区别，其实两者有着本质的区别，一个是传值参数，一个是传名参数，在调用boolAssert(5 > 3)时，5 > 3是已经计算出为true，然后传递给boolAssert方法，而myNameAssert(5 > 3)，表达式5 > 3没有事先计算好传递给myNameAssert，而是先创建一个函数类型的参数值，这个函数的apply方法将计算5 > 3，然后这个函数类型的值作为参数传给myNameAssert。

因此这两个函数一个明显的区别是，如果设置assertionsEnabled 为false，然后试图计算 x / 0 == 0：
```scala
    var assertionsEnabled = false
    val x = 5
    boolAssert( x / 0 == 0) // 报错 java.lang.ArithmeticException: / by zero
    myNameAssert ( x / 0 == 0) // 不报错，因为没有调用predicate，则x / 0 == 0 没有执行

    def myAssert(predicate: => Boolean ) = {
        if(assertionsEnabled && !predicate)
        throw new AssertionError
    }

    def boolAssert(predicate: Boolean ) = {
        if(assertionsEnabled && !predicate)
        throw new AssertionError
    }
```
可以看到boolAssert 抛出 java.lang.ArithmeticException: / by zero 异常，这是因为这是个传值参数，首先计算 x / 0 ，而抛出异常，而 myNameAssert 没有任何显示，这是因为这是个传名参数，传入的是一个函数类型的值，不会先计算x / 0 == 0,而在myNameAssert 函数体内，由于assertionsEnabled为false，传入的predicate没有必要计算(短路计算），因此什么也不会打印。

### => Unit 与 () =>Unit的区别
简单来说, => Unit是 传名函数, 只传入了一个表达式, 在调用时才会去执行, 使用 code调用

() => 是传值函数, 传入的计算后的值, 使用 code() 调用

### 自定义while循环
我们使用的while循环结构如下：
```scala
    while(/*条件*/){
      // 代码块
    }
```

我们可以把它看做为一个有两个参数的函数，即while(flag:Boolean)(block: => Unit)，具体实现如下：
```scala
object note {
  def main(args: Array[String]): Unit = {
    var i = 0;

    mywhile(i < 10) {
      println(i)
      i += 1
    }
    /* 上面那个循环等价于：

    mywhile(i < 10)(f)
    def f() = {
      println(i)
      i += 1
    }
    */

    def mywhile(flag: Boolean)(block: => Unit): Unit = {
      if (flag) { // 如果flag为真则执行代码块（代码块就是一个函数），并递归调用自己，实现循环的功能
        block
        mywhile(flag)(block)
      } // 如果flag为假，则不执行代码块，并停止循环，即停止递归
    }
  }
}
```

我们希望它打印0到9，但是事实并不如此，因为传flag参数的时候是用的传值调用，即一开始值就算完了是（0 < 10 ： true），这会导致递归会一直进行下去，解决这个问题很简单，把flag改成传名调用就行了，如下：
```scala
object note {
  def main(args: Array[String]): Unit = {
    var i = 0;
    mywhile(i < 10) {
      println(i)
      i += 1
    }

    def mywhile(flag: => Boolean)(block: => Unit): Unit = {
      if (flag) {
        block
        mywhile(flag)(block)
      }
    }
  }
}
 def mywhile(flag: => Boolean)(block: => Unit): Unit = {
      if (flag) {
        block
        mywhile(flag)(block)
      }
    }
  }
}

```

## 名字调用和值调用(实现自动资源管理)
摘抄自《scala程序设计（第2版）》————3.10名字调用和值调用

上一小节讲解异常处理实现的代码：
```scala
// src/main/scala/progscala2/rounding/TryCatch.scala
package progscala2.rounding

object TryCatch {
  /** Usage: scala rounding.TryCatch filename1 filename2 ... */
  def main(args: Array[String]): Unit = {
    args foreach (arg => countLines(arg))                            // <1>
  }

  import scala.io.Source                                             // <2>
  import scala.util.control.NonFatal

  def countLines(fileName: String) = {                               // <3>
    println()  // Add a blank line for legibility
    var source: Option[Source] = None                                // <4>
    try {                                                            // <5>
      source = Some(Source.fromFile(fileName))                       // <6>
      val size = source.get.getLines.size
      println(s"file $fileName has $size lines")
    } catch {
      case NonFatal(ex) => println(s"Non fatal exception! $ex")      // <7>
    } finally {
      for (s <- source) {                                            // <8>
        println(s"Closing $fileName...")
        s.close
      }
    }
  }
}
```
自动资源管理是一类常见的 Scala 设计模式。为了实现自动资源管理,Joshua Suereth 编写了一个名为ScalaARM(http://jsuereth.com/scala-arm/)的独立项目。下面让我们尝试编写自动资源管理程序。

我们曾动手实现了可重用资源管理器,其具体实现如下:
```scala
// src/main/scala/progscala2/rounding/TryCatchArm.scala
package progscala2.rounding
import scala.language.reflectiveCalls
import scala.util.control.NonFatal

// DeanW (Dec. 21, 2015): Refined the implementation and the usage
// example below to more clearly indicate the handling of the returned
// object of type T.
object manage {
  def apply[R <: { def close():Unit }, T](resource: => R)(f: R => T): T = {
    var res: Option[R] = None
    try {
      res = Some(resource)         // Only reference "resource" once!!
      f(res.get)                   // Return the T instance
    } catch {
      case NonFatal(ex) =>
        println(s"manage.apply(): Non fatal exception! $ex")
        throw ex
    } finally {
      if (res != None) {
        println(s"Closing resource...")
        res.get.close
      }
    }
  }
}

object TryCatchARM {
  /** Usage: scala rounding.TryCatch filename1 filename2 ... */
  def main(args: Array[String]): Unit = {
    val sizes = args map (arg => returnFileLength(arg))
    println("Returned sizes: " + (sizes.mkString(", ")))
  }

  import scala.io.Source

  def returnFileLength(fileName: String): Int = {
    println()  // Add a blank line for legibility
    manage(Source.fromFile(fileName)) { source =>
      val size = source.getLines.size
      println(s"file $fileName has $size lines")
      if (size > 200) throw new RuntimeException(s"Big file: $fileName!")
      size
    }
  }
}
```
能够将关注点分离(separation of concern,SOC)固然很好,不过为了实现这点,我们需要运用一些强大的新工具。

首先,我们将对象命名为 manage ,而不是 Manage 。通常我们都遵循类型名称首字母大写的规范,不过由于该示例使用 manage 的方式与使用函数的方式相似,因此未遵循该规范。我们希望 manage 在用户代码中看上去像一个内置的操作符,整个代码看起来就像是一个while 循环。我们可以在 countLines 方法中查看 manage 对象的使用方式。这个示例也演示了如何使用 Scala 工具构建小型领域特定语言(DSL)。

**manage.apply 方法**

manage.apply 方法声明看上去非常奇怪,为了能够理解该声明,我们将对其进行分解。下面再次列出了该方法的签名,我们将分行显示方法签名,而每一行也都提供了对应的注释。

```scala
def apply[
  R <: { def close():Unit },  ➊
  T ]                         ➋
  (resource: => R)            ➌
  (f: R => T) = {...}         ➍
```
➊ 这行出现了两个新的事物。 R 表示我们将要管理的资源类型。而 <: 则意味着 R 属于某其他类型的子类。在本例中 R 的父类型是一个包含 close():Unit 方法的结构类型。为了能帮助你更直观的理解 R 类型,尤其是当你之前没接触过结构化类型时,你可以认为 R <: Closable 表示 Closable 接口中定义了 close():Unit 方法并且 R 实现了Closable 接口。不过结构化类型允许我们使用反射机制嵌入包含 close():Unit 方法的任意类型(如 Source 类型)。反射机制会造成许多系统开销,而结构化类型代价也较为昂贵, 因此就像后缀表达式那样,Scala 将反射列为可选特性,为了能够让编译器相信我们知道我们在做什么,需要在代码中添加 import 语句。  
➋ 我们传入了用于处理资源的匿名函数,而 T 表示该匿名函数的返回值。  
➌ 尽管看上去像是一个声明体较为奇特的函数,但 resource 事实上是一个传名参数(by-name parameter)。我们暂且将其视为一个在调用时应省略括号的函数。  
➍ 最后我们将传入第二个参数列表,其中包含了一个输入为 resource 、返回值类型为 T的匿名函数,该匿名函数将负责处理 resource 。  

我们再回到注释 1,假设所有资源均实现了 Closable 抽象,那么 apply 方法声明看起来会
是下面这个样子:
```scala
object manage {
    def apply[ R <: Closable, T](resource: => R)(f: R => T) = {...}
    ...
}
```

resource 只 会 在 val res = Some(resource) 这 行 代 码 中 被 求 值, 这 行 代 码 必 不 可 少的。由于 resource 的表现与函数相似,因此就像是一个会被重复调用的函数,每次引用该变量时便会对其求值。但我们并不希望每次引用 resource 时都会执行一次 Source.fromFile(fileName) ,因为这意味着我们每次都会重新打开一个新的 Source 实例。

之后,我们将 res 值传入工作函数 f 中。

TryCatchARM.countLines 又是如何使用 manage 对象的呢? manage 对象在这段代码中看上去就像是一个 Scala 自带的控制结构,该控制结构包含了两个参数列表:一个用于创建Source 对象,而另一个则是处理 Source 对象的代码块。这样一来, manage 对象看起来就像是一个普通 while 语句了。

我们再回顾一下之前的代码,创建 Source 对象的第一条表达式其实并没有立刻执行,在进入 manage 对象之前,该表达式都没有被执行。直到执行 manage 对象内的代码 val res =Some(resource) 时,该表达式才会执行。这便是传名参数 resource 能提供的功能。我们编写的 manage.apply 方法可以接受任意表达式输入,但这些表达式将延后执行。

与 大 多 数 语 言 相 似,Scala 通 常 使 用 按 值 调 用(call-by-value) 的 语 法。 如 果manage(Source.fromFile(fileName)) 所处上下文遵循按值调用的方式的话,那么 Scala 将执行 Source.fromFile 方法,并将返回值传递给 manage 对象

通过将 Source.fromFile 推迟到 apply 方法中的代码行 val res = Some(resource) ,该行代
码等效于下列代码:
```scala
val res = Some(Source.fromFile(fileName))
```

正是因为要支持像延迟计算这样的语法,Scala 才提供了传名参数。

假如 Scala 未提供传名参数,该怎么办呢?我们可以使用匿名函数实现延迟计算,不过这种实现方式看起来略显丑陋。

对 manage 对象的调用代码看上去像是下列代码:
```scala
manage(() => Source.fromFile(fileName)) { Source =>
```
而在 apply 方法体中,将以函数调用的方式来引用 resource 。
```scala
val res = Some(resource())
```

尽管这种函数调用并不会给我们造成可怕的后果,不过像 manage 对象那样,我们也可以通过按名调用(call by name)构建自己的控制结构。

请记住,传名参数的行为与函数相似;每次使用该参数时便会执行表达式。在我们的ARM 示例里,我们希望该表达式只会被执行一次,但这并不能反映通常的情况。即可以被多次调用。

下面的示例中通过定义 continue 结构,实现了一个简单的类似于 while 循环的结构体:
```scala
// src/main/scala/progscala2/rounding/call-by-name.sc

@annotation.tailrec                                                  // <1>
def continue(conditional: => Boolean)(body: => Unit) {               // <2>
  if (conditional) {                                                 // <3>
    body                                                             // <4>
    continue(conditional)(body)
  }
}

var count = 0                                                        // <5>
continue(count < 5) {
  println(s"at $count")
  count += 1
}
```
➊ 确保了调用实现体时将采用尾递归的方式。  
➋ 定义 continue 函数,该函数接受两个参数列表:第一个列表中仅包含了一个传值参数conditional, 而第二个列表则包含了传值参数 body。body 代表了每次迭代都会执行的代码体  
➌ 检查当前是否满足条件。  
➍ 假如满足条件,将执行 body 参数,并递归调用 continue 函数。  
➎ 调用 continue 方法!  

读者需谨记一点:**传名参数会在每次被引用时求值**。(顺便提一下,上述实现描述了如何使用递归取代循环结构。)由于**传名参数的求值会被推迟,并可能会一再地被重复调用**,因此此类参数具有惰性。除此之外,Scala 也提供了惰性值(lazy value)。